In [1]:
import random
import copy
import numpy as np

def generate_random_points(n, m):
    return [[random.uniform(-10, 10) for _ in range(m)] for _ in range(n)]

def initialize_state(n, k):
    state = [random.randint(0, k-1) for _ in range(n)]
    return state

def calculate_objective(points, state, k):
    group_means = [[] for _ in range(k)]
    for i, group in enumerate(state):
        group_means[group].append(points[i])
    group_means = [np.sum(g, axis=0) / len(g) if len(g) > 0 else [0] * len(points[0]) for g in group_means]

    total_distance = 0
    for i, group in enumerate(state):
        point = points[i]
        mean = group_means[group]
        squared_distance = sum((p - m) ** 2 for p, m in zip(point, mean))
        total_distance += squared_distance

    return total_distance / len(state)

def generate_neighbors(state, k):
    neighbors = []
    for i in range(len(state)):
        for new_group in range(k):
            if new_group != state[i]:
                neighbor = copy.deepcopy(state)
                neighbor[i] = new_group
                neighbors.append(neighbor)
    return neighbors

def local_beam_search(points, k, beam_width):
    beam = [initialize_state(len(points), k) for _ in range(beam_width)]
    print(beam)

    converged = False
    iteration = 0

    while iteration < 10:
        iteration += 1
        beam_scores = [calculate_objective(points, state, k) for state in beam]
        print(beam_scores)

        beam = [s for _, s in sorted(zip(beam_scores, beam))]

        if all(beam_scores[0] == score for score in beam_scores):
            converged = True
        else:
            neighbors = []
            for state in beam[:beam_width // 2]:
                neighbors.extend(generate_neighbors(state, k))

            neighbor_scores = [calculate_objective(points, neighbor, k) for neighbor in neighbors]
            beam = [s for _, s in sorted(zip(neighbor_scores + beam_scores, neighbors + beam))][:beam_width]

    return beam[0]

# Example usage
n = 100
m = 5
k = 5
beam_width = 10

points = generate_random_points(n, m)

print(points)

best_state = local_beam_search(points, k, beam_width)

print("Best clustering:")
distinct_groups = set()
for i, group in enumerate(best_state):
    if group not in distinct_groups:
        distinct_groups.add(group)
        group_points = [j for j, g in enumerate(best_state) if g == group]
        print(f"Group {group}: {group_points}")


[[-0.5733955422307506, -7.881513459141843, -1.9230216066117585, -5.843559187962287, 1.15918775316581], [6.321589712476555, -0.19908793840967043, 7.942966982093903, 4.813414760309849, 3.1331574289498967], [-2.6151607450675662, 6.329044464908307, -9.134957968925884, 9.318424120150478, -7.946556392065858], [9.583273884069303, 5.48359350954177, 9.210385536543527, 1.0259116102983548, -8.442199045232856], [-7.181740300079545, -2.9698413938355444, 2.2341758358588066, 2.464619427677345, 0.539878642808711], [-5.967759233236867, 2.7909097136416356, 6.7547237554636865, -2.19375265459621, 2.418396167384298], [0.9180508522250026, -5.139908552651901, 9.249218535205017, -5.862896521447752, 8.97101706064791], [-7.188338459635419, -3.5753072152463705, 9.109872860080301, -8.08172687073245, 6.816708972830938], [1.977165268082155, 1.0912036088757144, 2.68784450796203, -4.423022592582178, 9.972619985649388], [8.310372823366809, 9.202147047534645, -1.4603433425589074, -8.020398490176355, -8.988159963991391]